# Visualizing Open Source Acoustic Metrics and Their Relationship to Satellite Oceanography: Part 2 (R)

## Emily Speciale, Adrienne Copeland, Carrie Wall Bell

This script outlines how to take the .csv data we extracted from Part 1 compare it to satellite oceanography data sets using plots and statistical analysis. We recommend using RStudio for this procedure.

First, we must download our satellite data of interest. In this module, we will download chlorophyll a and sea surface temperature (SST) data from June 2019, since June spanned the majority of the EX1903 cruise. We will be using NASA MODIS-Aqua data, which can be accessed from the link https://oceancolor.gsfc.nasa.gov/l3/.

To download the chlorophyll a data, click "Extract or Download Data". Change the Product to "Chlorphyll concentration", the Period to "Monthly", the Start Date to "2019-06-01", and the End Date to "2019-06-30". For Type, select Binned and Mapped. Then select "Extract". Because we are only looking at the southeastern U.S., we put our coordinates at 39 N, 24 S, -82 W, -72 E. Then we can review the order and extract the data. Upon recieving the data, simply download it onto your computer. *You will need to make an account with NASA Earth Data to extract and download the data. Once you create the account, the data will eventually load in your NASA Earth Data portal. 

Complete the same download procedure except for SST, select "Sea Surface Temperature (11u daytime) as the Product. 

With both our cruise data and satellite data downloaded, we can begin the analysis. 

In [ ]:
install.packages("oceanmap")
install.packages("ncdf4")
install.packages("raster")
install.packages("viridis")
install.packages("ggplot2")
install.packages("dplyr")
install.packages("sf")
install.packages("tidyverse")
install.packages("spData")
install.packages("oce")
install.packages("dplyr")
install.packages("lubridate")
install.packages("ggpubr")
install.packages("mgcv")

In [ ]:
library(oceanmap)
library(ncdf4)
library(raster)
library(viridis)
library(ggplot2)
library(dplyr)
library(sf)
library(tidyverse)
library(spData)
library(oce)
library(dplyr)
library(lubridate)
library(ggpubr)
library(mgcv)

## Downloading and Processing Satellite Data

The NASA MODIS data is processed as a raster, and we want to turn it into a data frame that we can easily extract our data from. First we will handle the chlorophyll a data.

In [ ]:
# Open our chlorophyll satellite data.
chl <- raster("/Volumes/Emily_Passport/monthly_chlorophyll/A20191522019181.L3m_MO_CHL.x_chlor_a.nc")

# Turn the chlorophyll raster data into a data frame so we can plot it. 
chl.df <- raster::as.data.frame(chl, xy = TRUE)

# Change the name of the columns to Longitude, Latitude, and Chlorophyll_a.
names(chl.df)[names(chl.df) == "x"] <- "Longitude"
names(chl.df)[names(chl.df) == "y"] <- "Latitude"
names(chl.df)[names(chl.df) == "Chlorophyll.Concentration..OCI.Algorithm"] <- "Chlorophyll_a"

# Add a new column that calculates the log chlorophyll values. We want to use log chlorophyll because it has a 
# more normal distribution. We can check distributions using the hist() function in R. 
hist(chl.df$Chlorophyll_a)
chl.df$log_chlorophyll <- log(chl.df$Chlorophyll_a)
hist(chl.df$log_chlorophyll)

# Now we will define and plot the log chlorophyll. We will be using this for many other plots later. 
chlor_a <- ggplot() + geom_raster(data = chl.df, aes(x = Longitude, y = Latitude, fill = log_chlorophyll), 
    interpolate = FALSE) + geom_sf(data = spData::world, fill = "grey80", col = "black") 
    + coord_sf(xlim = c(-81.6, -73), ylim = c(24.7, 38)) + theme_bw() 
    + scale_fill_gradientn(name = "Chlorophyll_a", colours = oce::oceColors9A(120))
print(chlor_a)

Now we will go through the same steps but with the SST data.

In [ ]:
# Open our SST satellite data.
sst <- raster("/Volumes/Emily_Passport/monthly_sst/AQUA_MODIS.20190601_20190630.L3m.MO.SST.x_sst.nc")

# Turn the SST raster data into a data frame so we can plot it. 
sst.df <- raster::as.data.frame(sst, xy = TRUE)

# Change the name of the columns to Longitude, Latitude, and SST.
names(sst.df)[names(sst.df) == "x"] <- "Longitude"
names(sst.df)[names(sst.df) == "y"] <- "Latitude"
names(sst.df)[names(sst.df) == "Sea.Surface.Temperature"] <- "SST"

# Check that the SST distribution is normal.
hist(sst.df$SST)

# Now we will define and plot the SST. We will be using this for many other plots later. 
sst_plot <- ggplot() + geom_raster(data = sst.df, aes(x = Longitude, y = Latitude, fill = SST), interpolate = FALSE) 
    + geom_sf(data = spData::world, fill = "grey80", col = "black") + coord_sf(xlim = c(-81.6, -73), ylim = c(24.7, 38)) 
    + theme_bw() + scale_fill_gradientn(name = "SST (Celsius)", colours = oce::oceColors9A(120)) 
    + labs(y="Latitude", x = "Longitude")
print(sst_plot)

## Downloading and Cleaning Echometric Data

Now we are going to import and clean our echometric csv data. We want to make sure we have a relatively normal distribution for each echometric, and may have to transform some echometrics into log scale. We also want to clean our data for bad points, which include rows with NaN, rows where our centre of mass is greater than the range, or rows with an Sv greater than -55 dB. 

In [ ]:
# Open our echometrics data.
echometrics <- read.csv("/Volumes/Emily_Passport/cruise_data_final.csv")

# Right now the values in our Time column of our echometric data are just string, and we want to convert them 
# to time for plotting. 
echometrics$Time <- ymd_hms(echometrics$Time)

# We need to check the distributions of our echometrics by creating histograms for them.
hist(echometrics$Sv_Avg)
hist(echometrics$Centre_of_Mass)
hist(echometrics$Inertia)
hist(echometrics$Proportion_Occupied)
hist(echometrics$Aggregation_Index)
hist(echometrics$Equivalent_Area)

# We can see that the aggregation index and equivalent area are very skewed, so we will transform them to log scale.
# Check their histograms.
echometrics$log_ea <- log(echometrics$Equivalent_Area)
echometrics$log_ai <- log(echometrics$Aggregation_Index)
hist(echometrics$log_ai)
hist(echometrics$log_ea)

# Although equivalent area is bimodal, we will accept this for now, as it is better than a skew. Now we can clean
# up our data. First, remove any rows with an NaN.
echometrics <- na.omit(echometrics)

# Now delete rows where centre of mass is greater than the range. Range gives us the maximum depth from the transducer 
# whereas centre of mass gives us the average location of backscatter in the water column. If COM > range, then 
# that means seafloor backscatter was not completely removed and contributed highly to the data point. 
# We want to remove these inaccuracies.
echometrics <- echometrics[!(echometrics$Range<=echometrics$Centre_of_Mass),]

# Lastly, we will filter out rows in which Sv is greater than -55, as these data points were also highly affected
# by the seafloor noise. 
echometrics <- echometrics[echometrics$Sv_Avg<=-55,]

## Combining Satellite and Echometric Datasets

Now we want to combine our echometrics data set with our satellite data values, based on latitude and longitude. We do this using our raster data. We start with chlorophyll.

In [ ]:
# First we convert our data into spatial objects.
echometrics_points <- SpatialPoints(echometrics[c("Longitude", "Latitude")], 
    proj4string = CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0"))
    
# Then we overlay coordinates onto our chlorophyll raster.
chlor_ras <- raster::extract(chl, echometrics_points, df = T, sp = T)

# Now we clean up the data frames, combine together, and then add it back into 
# our original data frame. 
chlor_bind <- bind_cols(as.data.frame(chlor_ras@data), as.data.frame(chlor_ras@coords))
echometrics <- full_join(echometrics, chlor_bind)

# We change the long preset name to Chlorphyll_a and then make a column for the log chlorophyll
# values. 
names(echometrics)[names(echometrics) == "Chlorophyll.Concentration..OCI.Algorithm"] <- "Chlorophyll_a"
echometrics$log_chlorophyll <- log(echometrics$Chlorophyll_a)

Complete the same procedure for SST. 

In [ ]:
# We already created our echometrics spatial objects so we can go straight to overlaying 
# the coordinates onto our SST raster.
sst_ras <- raster::extract(sst, echometrics_points, df = T, sp = T)

# Now we clean, combine, and add it back.
sst_bind <- bind_cols(as.data.frame(sst_ras@data), as.data.frame(sst_ras@coords))
echometrics <- full_join(echometrics, sst_bind)

# Finally, changing the new column name to SST.
names(echometrics)[names(echometrics) == "Sea.Surface.Temperature"] <- "SST"

## Plotting Echometrics vs. Timespan of Cruise

The first plots we will make will be each echometric vs. the timespan of the cruise. We will use a line plot, though you could also use a scatterplot to represent this data. 

In [ ]:
# Sv vs. Time
sv_time <- ggplot(data = echometrics, aes(x = Time, y = Sv_Avg)) + geom_line() + labs(y= "Sv (dB re 1 m^-1)", x = "Time") + theme_bw() 
print(sv_time)

# Centre of Mass vs. Time
com_time <- ggplot(data = echometrics, aes(x = Time, y = Centre_of_Mass)) + geom_line() + labs(y= "Centre of Mass (m)", x = "Time") + theme_bw() 
print(com_time)

# Inertia vs. Time
inertia_time <- ggplot(data = echometrics, aes(x = Time, y = Inertia)) + geom_line() + labs(y="Inertia (m^-2)", x = "Time") + theme_bw() 
print(inertia_time)

# Proportion Occupied vs. Time
po_time <- ggplot(data = echometrics, aes(x = Time, y = Proportion_Occupied)) + geom_line() + labs(y="Proportion Occupied", x = "Time") + theme_bw() 
print(po_time)

# Aggregation vs. Time
ai_time <- ggplot(data = echometrics, aes(x = Time, y = log_ai)) + geom_line() + labs(y="Log Index Aggregation", x = "Time") + theme_bw()
print(ai_time)

# Equivalent Area vs. Time 
ea_time <- ggplot(data = echometrics, aes(x = Time, y = log_ea)) + geom_line() + labs(y="Log Equivalent Area", x = "Time") + theme_bw()
print(ea_time)

# Putting all six plots on the same figure.
figure_1 <- ggarrange(sv_time, com_time, inertia_time, po_time, ai_time, ea_time, 
                      labels = c("A", "B", "C", "D", "E", "F"),
                      align = c("v"),
                      ncol = 2, nrow = 3) 
print(figure_1)

## Plotting Echometrics vs. Geographic Coordinates

The second plots we will make will represent the echometrics and their coordinates, before the addition of satellite data. 

In [ ]:
# First we must make a blank map to plot the echometrics on.
map <- ggplot() + geom_sf(data = spData::world, fill = "grey80", col = "black") + coord_sf(xlim = c(-81.6, -73), ylim = c(24.7, 38)) + theme_bw() + labs(y = "Latitude", x = "Longitude")
print(map)

# Sv vs. Coordinates
sv_position <- map + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Sv_Avg)) + scale_color_viridis(name = "Sv (dB re 1 m^-1)", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(sv_position)

# Centre of Mass vs. Coordinates
com_position <- map + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Centre_of_Mass)) + scale_color_viridis(name = "Centre of Mass (m)", discrete = FALSE, option = "A", direction = -1) + labs(y="Latitude", x = "Longitude")
print(com_position)

# Inertia vs. Coordinates
inertia_position <- map + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Inertia)) + scale_color_viridis(name = "Inertia (m^-2)", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(inertia_position)

# Proportion Occupied vs. Coordinates
po_position <- map + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Proportion_Occupied)) + scale_color_viridis(name = "Proportion Occupied", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(po_position)

# Aggregation vs. Coordinates
ai_position <- map + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = log_ai)) + scale_color_viridis(name = "Log Index Aggregation", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(ai_position)

# Equivalent Area vs. Coordinates
ea_position <- map + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = log_ea)) + scale_color_viridis(name = "Log Equivalent Area", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(ea_position)

# Putting all six plots on the same figure.
figure_2 <- ggarrange(sv_position, com_position, inertia_position, po_position, ai_position,
                      ea_position, labels = c("A", "B", "C", "D", "E", "F"),
                      align = c("hv"),
                      ncol = 3, nrow = 2)
print(figure_2)

## Plotting Echometrics and Chlorophyll-a Concentrations on Coordinates

Now we will plot the log of chlorophyll-a concentrations and echometric values on the same map. We will be using the chlorophyll map we set up previously to do this. 

In [ ]:
# Sv vs. Chlorophyll-a vs. Coordinates
sv_chlor <- chlor_a + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Sv_Avg)) + scale_color_viridis(name = "Sv (dB re 1 m^-1)", discrete = FALSE, option = "A") + labs(y = "Latitude", x = "Longitude")
print(sv_chlor)

# Centre of Mass vs. Chlorophyll-a vs. Coordinates
com_chlor <- chlor_a + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Centre_of_Mass)) + scale_color_viridis(name = "Centre of Mass (m)", discrete = FALSE, option = "A", direction = -1) + labs(y="Latitude", x = "Longitude")
print(com_chlor)

# Inertia vs. Chlorophyll-a vs. Coordinates
inertia_chlor <- chlor_a + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Inertia)) + scale_color_viridis(name = "Inertia (m^-2)", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(inertia_chlor)

# Proportion Occupied vs. Chlorophyll-a vs. Coordinates
po_chlor <- chlor_a + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Proportion_Occupied)) + scale_color_viridis(name = "Proportion Occupied", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(po_chlor)

# Aggregation vs. Chlorophyll-a vs. Coordinates
ai_chlor <- chlor_a + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = log_ai)) + scale_color_viridis(name = "Log Index Aggregation", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(ai_chlor)

# Equivalent Area vs. Chlorophyll-a vs. Coordinates
ea_chlor <- chlor_a + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = log_ea)) + scale_color_viridis(name = "Log Equivalent Area ", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(ea_chlor)

# Putting all six plots on the same figure.
figure_3 <- ggarrange(sv_chlor, com_chlor, inertia_chlor, po_chlor, ai_chlor, ea_chlor,
                      labels = c("A", "B", "C", "D", "E", "F"),
                      align = c("hv"),
                      ncol = 3, nrow = 2)
print(figure_3)

## Plotting Echometrics and SST on Coordinates

Now we will plot the SST and echometric values on the same map. We will be using the SST map we set up previously to do this. 

In [ ]:
# Sv vs. SST vs. Coordinates
sv_sst <- sst_plot + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Sv_Avg)) + scale_color_viridis(name = "Sv (dB re 1 m^-1)", discrete = FALSE, option = "A") + labs(y = "Latitude", x = "Longitude")
print(sv_sst)

# Centre of Mass vs. SST vs. Coordinates
com_sst <- sst_plot + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Centre_of_Mass)) + scale_color_viridis(name = "Centre of Mass (m)     ", discrete = FALSE, option = "A", direction = -1) + labs(y="Latitude", x = "Longitude")
print(com_sst)

# Inertia vs. SST vs. Coordinates
inertia_sst <- sst_plot + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Inertia)) + scale_color_viridis(name = "Inertia (m^-2)   ", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(inertia_sst)

# Proportion Occupied vs. SST vs. Coordinates
po_sst <- sst_plot + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = Proportion_Occupied)) + scale_color_viridis(name = "Proportion Occupied", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(po_sst)

# Aggregation vs. SST vs. Coordinates
ai_sst <- sst_plot + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = log_ai)) + scale_color_viridis(name = "Log Index Aggregation", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(ai_sst)

# Equivalent Area vs. SST vs. Coordinates
ea_sst <- sst_plot + geom_point(data = echometrics, aes(x = Longitude, y = Latitude, color = log_ea)) + scale_color_viridis(name = "Log Equivalent Area   ", discrete = FALSE, option = "A") + labs(y="Latitude", x = "Longitude")
print(ea_sst)

# Putting all six plots on the same figure.
figure_4 <- ggarrange(sv_sst, com_sst, inertia_sst, po_sst, ai_sst, ea_sst,         
          labels = c("A", "B", "C", "D", "E", "F"),
          ncol = 3, nrow = 2)

print(figure_4)

## Generalized Additive Models (GAMs)

Although our plots are visually valuable, we need to run statistical tests and models in order to understand the correlations between our echometrics and the satellite-derived data. GAMs have smooth functions that allow for a better fit of the data compared to linear models. Our first step is to run the significance test 

In [ ]:
# Running our GAM statistical test on chlorophyll vs. echometrics. 
gam_chlor <- gam(log_chlorophyll ~ s(Sv_Avg) + s(Centre_of_Mass) + s(Inertia) + s(Proportion_Occupied) + s(log_ai) + s(log_ea), data = echometrics, method = "REML")
summary(gam_chlor)

# Running our GAM statistical test on SST vs. echometrics.
gam_sst <- gam(SST ~ s(Sv_Avg) + s(Centre_of_Mass) + s(Inertia) + s(Proportion_Occupied) + s(log_ai) + s(log_ea), data=echometrics)
summary(gam_sst)


Before we continue, we need to run diagnostic plots to check that our data is appropriate for GAMs.

In [ ]:
par(mfrow = c(2,2))
gam.check(gam_chlor)

par(mfrow = c(2,2))
gam.check(gam_sst)

The diagnostic plots look okay so we can continue. 

In [ ]:
# We will plot the GAM for each echometric with chlorophyll.
par(mfrow = c(3,2))

# Chlorophyll vs. Sv GAM
plot(gam_chlor, select = 1, residuals = T, shade = TRUE, col = "black", xlab = "Sv (dB re 1 m^-1)", ylab = "Linear Predictor")

# Chlorophyll vs. Centre of Mass GAM
plot(gam_chlor, select = 2, residuals = T, shade = TRUE, col = "black", xlab = "Centre of Mass (m)", ylab = "Linear Predictor")

# Chlorophyll vs. Inertia GAM
plot(gam_chlor, select = 3, residuals = T, shade = TRUE, color = "black", xlab = "Inertia (m^-2)", ylab = "Linear Predictor")

# Chlorophyll vs. Proportion Occupied GAM
plot(gam_chlor, select = 4, residuals = T, shade = TRUE, color = "black", xlab = "Proportion Occupied", ylab = "Linear Predictor")

# Chlorophyll vs. Aggregation GAM
plot(gam_chlor, select = 5, residuals = T, shade = TRUE, color = "black",  xlab = "Log Index Aggregation", ylab = "Linear Predictor")

# Chlorophyll vs. Equivalent Area GAM
plot(gam_chlor, select = 6, residuals = T, shade = TRUE, color = "black", xlab = "Log Equivalent Area", ylab = "Linear Predictor")

# Now the GAMs for SST.
par(mfrow = c(3,2))

# SST vs. Sv GAM
plot(gam_sst, select = 1, residuals = T, shade = TRUE, col = "black", xlab = "Sv (dB re 1 m^-1)", ylab = "Linear Predictor")

# SST vs. Centre of Mass GAM
plot(gam_sst, select = 2, residuals = T, shade = TRUE, col = "black", xlab = "Centre of Mass (m)", ylab = "Linear Predictor")

# SST vs. Inertia GAM
plot(gam_sst, select = 3, residuals = T, shade = TRUE, color = "black", xlab = "Inertia (m^-2)", ylab = "Linear Predictor")

# SST vs. Proportion Occupied GAM
plot(gam_sst, select = 4, residuals = T, shade = TRUE, color = "black", xlab = "Proportion Occupied", ylab = "Linear Predictor")

# SST vs. Aggregation GAM
plot(gam_sst, select = 5, residuals = T, shade = TRUE, color = "black",  xlab = "Log Index Aggregation", ylab = "Linear Predictor")

# SST vs. Equivalent Area GAM
plot(gam_sst, select = 6, residuals = T, shade = TRUE, color = "black", xlab = "Log Equivalent Area", ylab = "Linear Predictor")